# Application of MPBNs to the Tumour invasion model by Cohen et al. 2015

We use Most Permissive Boolean Networks to analyse the Boolean model published in
[(Cohen et al., 2015)](https://doi.org/10.1371/journal.pcbi.1004571)
which shows cell fate decision processes related to tumour development.

In [1]:
import ginsim
from colomoto_jupyter import tabulate

This notebook has been executed using the docker image `colomoto/colomoto-docker:2020-07-01`

In [2]:
import mpbn

## Model setup
### Original model

In [3]:
lrg = ginsim.load("https://zenodo.org/record/3719029/files/SuppMat_Model_Master_Model.zginml?download=1")
ginsim.show(lrg)

Downloading https://zenodo.org/record/3719029/files/SuppMat_Model_Master_Model.zginml?download=1

### Conversion to MPBN

In [4]:
bn = ginsim.to_minibn(lrg)
mbn = mpbn.MPBooleanNetwork(bn)
len(mbn) # display number of nodes

32

### Input and output nodes

In [5]:
inputs = ["DNAdamage", "ECMicroenv"]
outputs = ["Apoptosis","CellCycleArrest","EMT","Invasion","Migration","Metastasis"]

## Analysis of reachable attractors

### Complete listing of attractors
Let us first compute the full list of attractors of the MPBN.

In [6]:
%time a = list(mbn.attractors())
tabulate(a)

CPU times: user 5.79 ms, sys: 26 µs, total: 5.81 ms
Wall time: 5.55 ms


,AKT1,AKT2,Apoptosis,CDH1,CDH2,CTNNB1,CellCycleArrest,DKK1,DNAdamage,ECMicroenv,EMT,ERK,GF,Invasion,Metastasis,Migration,NICD,SMAD,SNAI1,SNAI2,TGFbeta,TWIST1,VIM,ZEB1,ZEB2,miR200,miR203,miR34,p21,p53,p63,p73
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1
8,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0
1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1
7,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0
5,0,1,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0
6,0,1,0,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0
4,0,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0


Remark they are all fixed points and thus match exactly with the attractors using asynchronous BNs.

### Rechable attractors

Starting from all nodes but inputs to 0, we are interested in the set of reachable attractors, whatever the value of input nodes.

In [7]:
initial_condition = dict([(n,0) for n in bn if n not in inputs])

#### Wild-type
Let us first compute the reachable attractors in the wild type model. They match with the full attract listing.

In [8]:
%time a = list(mbn.attractors(reachable_from=initial_condition))
tabulate(a, columns=outputs)

CPU times: user 8.32 ms, sys: 0 ns, total: 8.32 ms
Wall time: 8.06 ms


,Apoptosis,CellCycleArrest,EMT,Invasion,Migration,Metastasis
0,0,0,0,0,0,0
3,0,1,1,0,0,0
1,0,1,1,1,1,1
5,1,1,0,0,0,0


#### Notch GoF mutant
Now, we apply the Notch gain of function, and predict, as MaBoSS with asynchronous BNs, the loss of apoptotic attractor reachability.

In [9]:
mbn_mut = mbn.copy()
mbn_mut["NICD"] = 1
%time a = list(mbn_mut.attractors(reachable_from=initial_condition))
tabulate(a, columns=outputs)

CPU times: user 7.83 ms, sys: 0 ns, total: 7.83 ms
Wall time: 7.58 ms


,Apoptosis,CellCycleArrest,EMT,Invasion,Migration,Metastasis
4,0,1,1,0,0,0
8,0,1,1,1,0,0
0,0,1,1,1,1,1


#### Notch GoF/p53 LoF double mutant
Adding p53 loss of function, leads, as MaBoSS, only to the metastatic attractor.

In [10]:
mbn_mut["p53"] = 0
%time a = list(mbn_mut.attractors(reachable_from=initial_condition))
tabulate(a, columns=outputs)

CPU times: user 7.15 ms, sys: 0 ns, total: 7.15 ms
Wall time: 6.87 ms


,Apoptosis,CellCycleArrest,EMT,Invasion,Migration,Metastasis
0,0,1,1,1,1,1
